# Lite Notebook · openai/gpt-oss-20b · Beginner

Token‑light tutorial: environment setup + runnable calls. Uses OpenAI SDK against selected provider (Poe/OpenAI‑compatible/local).

Details:
- Provider: poe
- Model: gpt-oss-20b

## Learning Objectives

- Configure provider and API key correctly
- Run a model call with safe defaults
- Tune basic parameters and/or streaming
- Record simple telemetry or ranking step

## Step 1: Environment & Client Setup

Configure an OpenAI-compatible client pointed at Poe and confirm the model you plan to use.

In [ ]:
# Step 1: Environment & Client Setup
# Offline-friendly: gate live API calls behind RUN_LIVE.

from __future__ import annotations

import os
import random
import subprocess
import sys
from importlib import import_module
from pathlib import Path


RUN_LIVE = os.getenv("RUN_LIVE", "0").strip() in ("1", "true", "True")
# Determinism knobs (used by quality gate and examples)
SEED = int(os.getenv("SEED", "42"))
random.seed(SEED)
try:
    import numpy as _np  # type: ignore
except Exception as _e:  # pragma: no cover
    _np = None
if _np is not None:
    _np.random.seed(SEED)

def _ensure_package(module: str, pip_name: str | None = None) -> None:
    # Install a module on demand to keep the notebook self-contained.
    try:
        import_module(module)
    except Exception:  # pragma: no cover
        if not RUN_LIVE:
            return
        subprocess.check_call([sys.executable, "-m", "pip", "install", pip_name or module])


_ensure_package("dotenv", "python-dotenv")

try:
    from dotenv import load_dotenv
except ImportError:
    load_dotenv = None


def _ingest_env_file(path: Path) -> None:
    # Load key=value pairs without overwriting existing values.
    if load_dotenv:
        load_dotenv(path, override=False)
        return
    if not path.exists():
        return
    for line in path.read_text(encoding="utf-8").splitlines():
        stripped = line.strip()
        if not stripped or stripped.startswith("#") or "=" not in stripped:
            continue
        key, value = stripped.split("=", 1)
        os.environ.setdefault(key.strip(), value.strip())


for candidate in (Path(".env"), Path(".env.local")):
    _ingest_env_file(candidate)

if os.getenv("POE_API_KEY") and not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = os.environ["POE_API_KEY"]

BASE_URL = os.environ.setdefault("OPENAI_BASE_URL", "https://api.poe.com/v1")
API_KEY = os.getenv("OPENAI_API_KEY")

MODEL_ID = "gpt-oss-20b"

if RUN_LIVE:
    _ensure_package("openai")
    from openai import OpenAI
    client = OpenAI(base_url=BASE_URL, api_key=API_KEY)
    if not API_KEY:
        raise RuntimeError("Set POE_API_KEY or OPENAI_API_KEY before continuing.")
else:
    # Deterministic stub client for smoke tests/offline execution.
    class _Msg:
        def __init__(self, content: str):
            self.content = content

    class _Choice:
        def __init__(self, content: str):
            self.message = _Msg(content)

    class _Reply:
        def __init__(self, content: str):
            self.choices = [_Choice(content)]
            class _U:
                'usage container'
            self.usage = _U()
            self.usage.prompt_tokens = 0
            self.usage.completion_tokens = 0

    class _DummyCompletions:
        def create(self, *, model: str, messages, **kwargs):  # type: ignore[no-untyped-def]
            if isinstance(messages, (list, tuple)):
                prompt = next(
                    (m.get("content", "") for m in messages if isinstance(m, dict) and m.get("role") == "user"),
                    "",
                )
            else:
                prompt = ""
            return _Reply("[SMOKE] Echo from " + str(model) + " → " + str(prompt))

    class _DummyChat:
        def __init__(self):
            self.completions = _DummyCompletions()

    class _DummyClient:
        def __init__(self, *a, **k):
            self.chat = _DummyChat()

    client = _DummyClient()

print(f"Base URL → {BASE_URL}")
print(f"Client ready for model → {MODEL_ID} (RUN_LIVE={RUN_LIVE})")
print(f"Seed → {SEED}")

## Step 2: First Chat Completion

Send your brief to the model with safe defaults and review the reply.

In [ ]:
# Step 2: First Chat Completion
# Draft a short helper that sends the user brief to the model once.

def _guardrail_block(prompt: str) -> None:
    # Optional guardrail: block risky prompts when enabled (ENABLE_GUARDRAILS=1).
    enabled = os.getenv("ENABLE_GUARDRAILS", "0").strip() in ("1", "true", "True")
    if not enabled:
        return
    text = (prompt or "").lower()
    risky = ("rm -rf" in text) or ("api key" in text) or ("password" in text)
    if risky:
        raise ValueError("Blocked by guardrails: prompt contains risky terms.")

def _show_request(params: dict) -> None:
    # Print a cURL-like view of the request for transparency.
    try:
        import json as _json
        payload = {
            "model": params["model"],
            "messages": params["messages"],
            "max_tokens": params.get("max_tokens", 256),
            "temperature": params.get("temperature", 0),
        }
        curl = (
            "curl -sS -X POST "
            + os.getenv("OPENAI_BASE_URL", "https://api.poe.com/v1") + "/chat/completions "
            + "-H 'Content-Type: application/json' "
            + "-H 'Authorization: Bearer $OPENAI_API_KEY' "
            + "-d '" + _json.dumps(payload).replace("'", "\'") + "'"
        )
        print("Show Request (cURL):")
        print(curl)
    except Exception:
        pass

def _cost_hint(model: str) -> str:
    # Minimal, static hints where known; otherwise N/A.
    table = {}
    return table.get(model, "N/A")

def run_once(prompt: str) -> str:
    # Call the Poe-hosted model with safe defaults and return the text.
    _guardrail_block(prompt)
    temperature = float(os.getenv("TEMP_OVERRIDE", os.getenv("TEMPERATURE", "0.6")))
    params = dict(
        model=MODEL_ID,
        messages=[
            {"role": "system", "content": "You are a patient teacher."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=320,
        temperature=temperature,
    )
    try:
        response = client.chat.completions.create(**params)
    except Exception as e:
        print(f"Live request failed: {e}")
        print("Show sample output:")
        return "[SAMPLE OUTPUT] This is a representative reply shown when the provider is unavailable."
    _show_request(params)
    choice = response.choices[0].message.content
    print(f"Prompt tokens → {getattr(response.usage, 'prompt_tokens', 'N/A')}, completion → {getattr(response.usage, 'completion_tokens', 'N/A')}")
    hint = _cost_hint(MODEL_ID)
    print(f"Cost hint → {hint}")
    return choice


first_reply = run_once('Smoke')
print("\nModel reply:\n")
print(first_reply)

## Step 3: Adjust Parameters

Contrast two temperatures to see how the narrative changes.

In [ ]:
# Step 3: Adjust Parameters
# Compare two temperatures to see how tone and verbosity change.

def compare_temperatures(prompt: str, temperatures: tuple[float, float]) -> None:
    for temp in temperatures:
        print(f"\n--- Temperature {temp} ---")
        reply = client.chat.completions.create(
            model=MODEL_ID,
            messages=[
                {"role": "system", "content": "You teach with concise, friendly explanations."},
                {"role": "user", "content": prompt},
            ],
            max_tokens=280,
            temperature=temp,
        )
        print(reply.choices[0].message.content.strip())


compare_temperatures('Smoke', (0.2, 0.8))

Note: Live API calls require OPENAI_API_KEY and OPENAI_BASE_URL to be set. The setup cell helps map keys for Poe or gateways.